In [13]:
# -*- coding:utf-8 -*-
'''
@Author: LLC

@Date: 2018/10/11

@Description: Use statistic method to predict the CTR.
    
'''

'\n@Author: LLC\n\n@Date: 2018/10/11\n\n@Description: Use statistic method to predict the CTR.\n    \n'

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Read Dataset:
Train, Test, Validation

In [5]:
test_df = pd.read_csv('..\data\oppo_train_valt_est_csv\oppo_round1_test_A_20180929.csv', encoding='UTF-8')
train_df = pd.read_csv('..\data\oppo_train_valt_est_csv\oppo_round1_train_20180929.csv', encoding='UTF-8')
vali_df = pd.read_csv('..\data\oppo_train_valt_est_csv\oppo_round1_vali_20180929.csv', encoding='UTF-8')

## Poking the Dataset

In [6]:
train_df.describe(include='all')

,prefix,sup_dict,title,type,click
count,2000000,2000000,1999987,2000000,2.000000e+06
unique,156960,152560,249833,22,NaN
top,汽车\t,"{""汽车报价"": ""0.010"", ""汽车标志大全"": ""0.020"", ""汽车之家"": ""...",汽车之家,百科,NaN
freq,20066,20066,34857,722096,NaN
mean,NaN,NaN,NaN,NaN,3.720975e-01
std,NaN,NaN,NaN,NaN,4.833643e-01
min,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,0.000000e+00
50%,NaN,NaN,NaN,NaN,0.000000e+00
75%,NaN,NaN,NaN,NaN,1.000000e+00


## Check the Nan Data

In [7]:
print(train_df.dtypes)
print(train_df.isna().sum())
print(test_df.isna().sum())
print(vali_df.isna().sum())

prefix      object
sup_dict    object
title       object
type        object
click        int64
dtype: object
prefix       0
sup_dict     0
title       13
type         0
click        0
dtype: int64
prefix      0
sup_dict    0
title       0
type        0
click       0
dtype: int64
prefix      0
sup_dict    0
title       0
type        0
click       0
dtype: int64


Delete the Nan Value in Title. 

In [8]:
# locs = np.where(train_df['title'].isna().values==True)
# train_df.iloc[locs[0], :]
train_df.dropna(subset = ['title'], inplace = True)
print(train_df.isna().sum())

prefix      0
sup_dict    0
title       0
type        0
click       0
dtype: int64


In [ ]:
# train_df_vehice.describe(include='all')

In [10]:
# locs = np.where(train_df['prefix']=='汽车\t')
# train_df_vehice = train_df.iloc[locs[0], :]
# train_df_vehice['assemble'] = train_df_vehice.loc[:, 'title'] + ' ' +  train_df_vehice.loc[:, 'type'] 

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# train_df_vehice['assemble'].value_counts()

# Get the Statistic Result

In [11]:
train_df['assemble'] = train_df['prefix'] + ' ' + train_df['title'] + ' ' + train_df['type']
train_df_click = train_df.loc[train_df['click']==1,:].groupby('assemble', as_index=False)
train_df_noclick = train_df.loc[train_df['click']==0,:].groupby('assemble', as_index=False)


In [45]:
histogram = train_df['assemble'].value_counts().values
# plt.plot(np.log(histogram))
# plt.show()

for i in [1,2,5,10,25,50,100]:
    num = i
    lower_num = histogram[np.where(histogram<=num)].sum()
    total_num = histogram.sum()
    print(i, 'th: ')
    print('lower_num: ', lower_num, 'total_num:', total_num)
    print('rate:', lower_num/total_num)
    print('\n')

1 th: 
lower_num:  177237 total_num: 1999987
rate: 0.08861907602399416


2 th: 
lower_num:  280645 total_num: 1999987
rate: 0.14032341210217866


5 th: 
lower_num:  465024 total_num: 1999987
rate: 0.2325135113378237


10 th: 
lower_num:  636405 total_num: 1999987
rate: 0.3182045683296941


25 th: 
lower_num:  898693 total_num: 1999987
rate: 0.449349420771235


50 th: 
lower_num:  1107301 total_num: 1999987
rate: 0.5536540987516418


100 th: 
lower_num:  1306876 total_num: 1999987
rate: 0.653442247374608




进行一些data frame的操作

In [12]:
click = train_df_click.count().loc[:, ['assemble', 'click']]
unclick = train_df_noclick.count().loc[:, ['assemble', 'click']]
result = pd.merge(click, unclick, how='outer', on=['assemble', 'assemble'])
result.rename(columns={'click_x':'click', 'click_y':'unclick'}, inplace = True)
result.fillna(0, inplace = True)
result['rate'] = result['click'] / (result['click'] + result['unclick'])

In [46]:
vali_df['assemble'] = vali_df['prefix'] + ' ' + vali_df['title'] + ' ' + vali_df['type']
vali_df

,prefix,sup_dict,title,type,click,assemble
0,重庆旅游\t,"{""重庆旅游景区"": ""0.018"", ""重庆旅游攻略"": ""0.373"", ""重庆旅游景点...",皇包车旅行,应用,0,重庆旅游\t 皇包车旅行 应用
1,婆婆来了\t,"{""婆婆来了大结局"": ""0.021"", ""婆婆来了电视剧"": ""0.100"", ""婆婆来了...",搜狐视频,应用,0,婆婆来了\t 搜狐视频 应用
2,张国荣\t,"{""张国荣遗体很恐怖"": ""0.020"", ""张国荣头像"": ""0.013"", ""张国荣24...",张国荣,歌手,0,张国荣\t 张国荣 歌手
3,陌陌\t,"{""陌陌晒奶"": ""0.008"", ""陌陌下载安装"": ""0.009"", ""陌陌交友"": ""...",陌陌,百科,0,陌陌\t 陌陌 百科
4,畅游\t,"{""畅游阁"": ""0.322"", ""畅游六零年代"": ""0.134"", ""畅游书城82212...",李畅游,百科,0,畅游\t 李畅游 百科
5,速腾\t,"{""速腾2018款"": ""0.008"", ""速腾1.6"": ""0.023"", ""速腾报价"":...",全新速腾,百科,0,速腾\t 全新速腾 百科
6,学位\t,"{""学位英语"": ""0.032"", ""学位证"": ""0.045"", ""学位网"": ""0.10...",学位证书编号怎么查,经验,1,学位\t 学位证书编号怎么查 经验
7,汽车\t,"{""汽车报价"": ""0.010"", ""汽车标志大全"": ""0.020"", ""汽车之家"": ""...",汽车之家,应用,0,汽车\t 汽车之家 应用
8,骏\t,"{""骏马图片"": ""0.016"", ""骏马奔驰保边疆"": ""0.040"", ""骏派a70"":...",骏派A50,汽车,1,骏\t 骏派A50 汽车
9,支\t,"{""支付宝下载"": ""0.012"", ""支气管炎"": ""0.010"", ""支付宝提现要手续费...",支付宝官网,网站,1,支\t 支付宝官网 网站
